In [1]:
import pandas as pd

# having the data in 2 files
# plan is to append the data to work with one datasource
fact_sales = pd.read_csv("/content/drive/MyDrive/aksziii_sales/fact_sales.csv")
fact_sales_2024 = pd.read_csv("/content/drive/MyDrive/aksziii_sales/fact_sales_ytd2024.csv")

fact_sales.head()

,month,store_id,product_id,sales_quantity,sales_value,discount
0,01/01/2021,1,1,221,427.856,56.420
1,01/01/2021,1,3,370,1120.360,4.050
2,01/01/2021,9,1,18,34.848,5.164
3,01/01/2021,9,3,7,21.196,0.000
4,01/01/2021,10,1,344,665.984,100.548


In [2]:
fact_sales_2024.head()

,month,product_id,st#00Moonshine#1_sys - sales_quantity,st#00Moonshine#1_sys - sales_value,st#00Moonshine#1_sys - discount,st#00Enroth#9_sys - sales_quantity,st#00Enroth#9_sys - sales_value,st#00Enroth#9_sys - discount,st#00Veilguard#10_sys - sales_quantity,st#00Veilguard#10_sys - sales_value,...,st#00Shelter#7_sys - discount,st#00Rose's Garden#18_sys - sales_quantity,st#00Rose's Garden#18_sys - sales_value,st#00Rose's Garden#18_sys - discount,st#00SofiaTech#8_sys - sales_quantity,st#00SofiaTech#8_sys - sales_value,st#00SofiaTech#8_sys - discount,st#00Twisting Nether#19_sys - sales_quantity,st#00Twisting Nether#19_sys - sales_value,st#00Twisting Nether#19_sys - discount
0,01/01/2024,_4#Magic Wand,1464.0,4187.040,598.322,169.0,483.340,67.070,3563.0,10190.180,...,292.024,400.0,1144.000,106.234,344.0,983.840,116.424,77.0,220.220,21.694
1,01/01/2024,_5#Magic Rod,3342.0,8054.220,681.722,226.0,544.660,75.714,6106.0,14715.460,...,461.588,1158.0,2790.780,209.904,1078.0,2597.980,130.928,364.0,877.240,60.358
2,01/01/2024,_6#Magic Rod,4874.0,18540.696,1506.494,290.0,1103.160,150.822,6563.0,24965.652,...,1116.636,1814.0,6900.456,504.368,2025.0,7703.100,347.586,278.0,1057.512,78.368
3,01/01/2024,_7#Magic Rod,1232.0,2224.992,0.000,60.0,108.360,0.000,1999.0,3610.194,...,0.000,NaN,NaN,NaN,49.0,88.494,0.000,42.0,75.852,0.000
4,01/01/2024,_10#Whistleblower,106.0,510.284,0.000,3.0,14.442,0.000,174.0,837.636,...,13.744,17.0,81.838,3.254,20.0,96.280,6.072,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# goal is to create the same table schema in both tables

# first extracting the 'product_id' between delimeters
fact_sales_2024['product_id'] = fact_sales_2024['product_id'].str.extract(r'^.*_(.*?)#')

# then unpivoting all columns except for 'month' and 'product_id' in the 'fact_sales_2024' table
unpivoted_fact_sales_2024 = pd.melt(fact_sales_2024, id_vars =['month', 'product_id'])

# now extracting the 'store_id' and 'variable' (measure name) from the column names
unpivoted_fact_sales_2024['store_id'] = unpivoted_fact_sales_2024['variable'].str.extract(r'^.*#(.*?)_')
unpivoted_fact_sales_2024['variable'] = unpivoted_fact_sales_2024['variable'].str.extract(r' - (.*)')

# removig null value records from data
# then restructuring the table to get the same layout as the 'fact_sales' table
unpivoted_fact_sales_2024 = unpivoted_fact_sales_2024.dropna(subset=['value'])
unpivoted_fact_sales_2024 = unpivoted_fact_sales_2024.pivot(index=['month', 'store_id', 'product_id'], columns='variable', values='value')
unpivoted_fact_sales_2024 = unpivoted_fact_sales_2024.reset_index()

unpivoted_fact_sales_2024.head()

variable,month,store_id,product_id,discount,sales_quantity,sales_value
0,01/01/2024,1,10,0.000,106.0,510.284
1,01/01/2024,1,101,7015.286,11065.0,53222.650
2,01/01/2024,1,102,834.192,1053.0,6018.948
3,01/01/2024,1,105,302.094,419.0,2143.604
4,01/01/2024,1,107,7.892,13.0,59.462


In [4]:
# appending 'fact_sales_2024' data to 'fact_sales' data
output = pd.concat([fact_sales, unpivoted_fact_sales_2024])

# 'month' data is stored as 1st day of month in the table
# realized that the number of month has been swapped with the number of day - let's fix that
output['month'] = pd.to_datetime(output['month'])
output['month'] = output['month'].apply(lambda x: x.replace(day=x.month, month=x.day))

# data type
output['store_id'] = output['store_id'].astype(int)
output['product_id'] = output['product_id'].astype(int)
output.dtypes



# extracting the output data to excel as 'fact_sales_total'
output.to_excel('/content/drive/MyDrive/aksziii_sales/fact_sales_total.xlsx', index=False)
output.head()

,month,store_id,product_id,sales_quantity,sales_value,discount
0,2021-01-01,1,1,221.0,427.856,56.420
1,2021-01-01,1,3,370.0,1120.360,4.050
2,2021-01-01,9,1,18.0,34.848,5.164
3,2021-01-01,9,3,7.0,21.196,0.000
4,2021-01-01,10,1,344.0,665.984,100.548
